# Unit commitment


This tutorial runs through examples of unit commitment for generators at a single bus. Examples of minimum part-load, minimum up time, minimum down time, start up costs, shut down costs and ramp rate restrictions are shown.

To enable unit commitment on a generator, set its attribute committable = True.

In [1]:
import pandas as pd

import pypsa

## Minimum part load demonstration

In final hour load goes below part-load limit of coal gen (30%), forcing gas to commit.

In [2]:
nu = pypsa.Network(snapshots=range(4))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    committable=True,
    p_min_pu=0.3,
    marginal_cost=20,
    p_nom=10000,
)

nu.add(
    "Generator",
    "gas",
    bus="bus",
    committable=True,
    marginal_cost=70,
    p_min_pu=0.1,
    p_nom=1000,
)

nu.add("Load", "load", bus="bus", p_set=[4000, 6000, 5000, 800])

In [3]:
nu.optimize()

Index(['bus'], dtype='object', name='Bus')
Index(['bus'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.solvers:Log file at /tmp/highs.log.
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 32 primals, 36 duals
Objective: 3.56e+05
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-com-p-lower, Generator-com-p-upper, Generator-com-transition-start-up, Generator-com-transition-shut-down, Generator-com-status-min_up_time_must_stay_up were not assigned to the network.


Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d-dirty]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
30 rows, 12 cols, 58 nonzeros
0 rows, 3 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve: Optimal

Solving report
  Status            Optimal
  Primal bound      356000
  Dual bound        356000
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    356000 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (total)
                    0.00 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)


('ok', 'optimal')

In [4]:
nu.generators_t.status

Generator-com,coal,gas
snapshot,,
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,0.0,1.0


In [5]:
nu.generators_t.p

Generator,coal,gas
snapshot,,
0,4000.0,0.0
1,6000.0,0.0
2,5000.0,0.0
3,0.0,800.0


## Minimum up time demonstration

Gas has minimum up time, forcing it to be online longer

In [6]:
nu = pypsa.Network(snapshots=range(4))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    committable=True,
    p_min_pu=0.3,
    marginal_cost=20,
    p_nom=10000,
)

nu.add(
    "Generator",
    "gas",
    bus="bus",
    committable=True,
    marginal_cost=70,
    p_min_pu=0.1,
    up_time_before=0,
    min_up_time=3,
    p_nom=1000,
)

nu.add("Load", "load", bus="bus", p_set=[4000, 800, 5000, 3000])

In [7]:
nu.optimize()

Index(['bus'], dtype='object', name='Bus')
Index(['bus'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.solvers:Log file at /tmp/highs.log.
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 32 primals, 39 duals
Objective: 3.06e+05
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-com-p-lower, Generator-com-p-upper, Generator-com-transition-start-up, Generator-com-transition-shut-down, Generator-com-up-time, Generator-com-status-min_up_time_must_stay_up were not assigned to the network.


Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d-dirty]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
33 rows, 16 cols, 73 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve: Optimal

Solving report
  Status            Optimal
  Primal bound      306000
  Dual bound        306000
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    306000 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (total)
                    0.00 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)


('ok', 'optimal')

In [8]:
nu.generators_t.status

Generator-com,coal,gas
snapshot,,
0,1.0,1.0
1,0.0,1.0
2,1.0,1.0
3,1.0,0.0


In [9]:
nu.generators_t.p

Generator,coal,gas
snapshot,,
0,3900.0,100.0
1,0.0,800.0
2,4900.0,100.0
3,3000.0,-0.0


## Minimum down time demonstration

Coal has a minimum down time, forcing it to go off longer.

In [10]:
nu = pypsa.Network(snapshots=range(4))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    committable=True,
    p_min_pu=0.3,
    marginal_cost=20,
    min_down_time=2,
    down_time_before=1,
    p_nom=10000,
)

nu.add(
    "Generator",
    "gas",
    bus="bus",
    committable=True,
    marginal_cost=70,
    p_min_pu=0.1,
    p_nom=4000,
)

nu.add("Load", "load", bus="bus", p_set=[3000, 800, 3000, 8000])

In [11]:
nu.optimize()

Index(['bus'], dtype='object', name='Bus')
Index(['bus'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.solvers:Log file at /tmp/highs.log.
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 32 primals, 40 duals
Objective: 4.86e+05
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-com-p-lower, Generator-com-p-upper, Generator-com-transition-start-up, Generator-com-transition-shut-down, Generator-com-down-time, Generator-com-status-min_up_time_must_stay_up, Generator-com-status-min_down_time_must_stay_up were not assigned to the network.


Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d-dirty]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
27 rows, 12 cols, 55 nonzeros
4 rows, 4 cols, 8 nonzeros
3 rows, 3 cols, 6 nonzeros
2 rows, 2 cols, 4 nonzeros
0 rows, 1 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve: Optimal

Solving report
  Status            Optimal
  Primal bound      486000
  Dual bound        486000
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    486000 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (total)
                    0.00 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)


('ok', 'optimal')

In [12]:
nu.objective

486000.0

In [13]:
nu.generators_t.status

Generator-com,coal,gas
snapshot,,
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0


In [14]:
nu.generators_t.p

Generator,coal,gas
snapshot,,
0,0.0,3000.0
1,0.0,800.0
2,3000.0,0.0
3,8000.0,0.0


## Start up and shut down costs

Now there are associated costs for shutting down, etc

In [15]:
nu = pypsa.Network(snapshots=range(4))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    committable=True,
    p_min_pu=0.3,
    marginal_cost=20,
    min_down_time=2,
    start_up_cost=5000,
    p_nom=10000,
)

nu.add(
    "Generator",
    "gas",
    bus="bus",
    committable=True,
    marginal_cost=70,
    p_min_pu=0.1,
    shut_down_cost=25,
    p_nom=4000,
)

nu.add("Load", "load", bus="bus", p_set=[3000, 800, 3000, 8000])

In [16]:
nu.optimize()

Index(['bus'], dtype='object', name='Bus')
Index(['bus'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.solvers:Log file at /tmp/highs.log.
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 32 primals, 39 duals
Objective: 4.91e+05
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-com-p-lower, Generator-com-p-upper, Generator-com-transition-start-up, Generator-com-transition-shut-down, Generator-com-down-time, Generator-com-status-min_up_time_must_stay_up were not assigned to the network.


Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d-dirty]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
33 rows, 23 cols, 78 nonzeros
14 rows, 12 cols, 29 nonzeros
8 rows, 9 cols, 17 nonzeros
6 rows, 8 cols, 13 nonzeros
6 rows, 7 cols, 12 nonzeros
Objective function is integral with scale 0.04

Solving MIP model with:
   6 rows
   7 cols (7 binary, 0 integer, 0 implied int., 0 continuous)
   12 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   341050          inf                  inf        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      491025
  Dual bound        491025
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    491025 (

('ok', 'optimal')

In [17]:
nu.objective

491025.0

In [18]:
nu.generators_t.status

Generator-com,coal,gas
snapshot,,
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0


In [19]:
nu.generators_t.p

Generator,coal,gas
snapshot,,
0,0.0,3000.0
1,0.0,800.0
2,3000.0,0.0
3,8000.0,0.0


## Ramp rate limits

In [20]:
nu = pypsa.Network(snapshots=range(6))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    marginal_cost=20,
    ramp_limit_up=0.1,
    ramp_limit_down=0.2,
    p_nom=10000,
)

nu.add("Generator", "gas", bus="bus", marginal_cost=70, p_nom=4000)

nu.add("Load", "load", bus="bus", p_set=[4000, 7000, 7000, 7000, 7000, 3000])

In [21]:
nu.optimize()

Index(['bus'], dtype='object', name='Bus')
Index(['bus'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.01s
INFO:linopy.solvers:Log file at /tmp/highs.log.
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 12 primals, 40 duals
Objective: 9.50e+05
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-fix-p-ramp_limit_up, Generator-fix-p-ramp_limit_down were not assigned to the network.


Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d-dirty]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
10 rows, 6 cols, 20 nonzeros
2 rows, 3 cols, 4 nonzeros
2 rows, 3 cols, 4 nonzeros
Presolve : Reductions: rows 2(-38); columns 3(-9); elements 4(-52)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          0     9.5000000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  9.5000000000e+05
HiGHS run time      :          0.00


('ok', 'optimal')

In [22]:
nu.generators_t.p

Generator,coal,gas
snapshot,,
0,4000.0,-0.0
1,5000.0,2000.0
2,6000.0,1000.0
3,7000.0,-0.0
4,5000.0,2000.0
5,3000.0,-0.0


In [23]:
nu = pypsa.Network(snapshots=range(6))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    marginal_cost=20,
    ramp_limit_up=0.1,
    ramp_limit_down=0.2,
    p_nom_extendable=True,
    capital_cost=1e2,
)

nu.add("Generator", "gas", bus="bus", marginal_cost=70, p_nom=4000)

nu.add("Load", "load", bus="bus", p_set=[4000, 7000, 7000, 7000, 7000, 3000])

In [24]:
nu.optimize()

Index(['bus'], dtype='object', name='Bus')
Index(['bus'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.solvers:Log file at /tmp/highs.log.
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 13 primals, 41 duals
Objective: 1.68e+06
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Generator-ext-p-ramp_limit_up, Generator-ext-p-ramp_limit_down were not assigned to the network.


Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d-dirty]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
16 rows, 7 cols, 42 nonzeros
14 rows, 6 cols, 36 nonzeros
Presolve : Reductions: rows 14(-27); columns 6(-7); elements 36(-37)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          6     1.6750000000e+06 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 6
Objective value     :  1.6750000000e+06
HiGHS run time      :          0.00


('ok', 'optimal')

In [25]:
nu.generators.p_nom_opt

Generator
coal    5000.0
gas     4000.0
Name: p_nom_opt, dtype: float64

In [26]:
nu.generators_t.p

Generator,coal,gas
snapshot,,
0,4000.0,-0.0
1,4500.0,2500.0
2,5000.0,2000.0
3,5000.0,2000.0
4,4000.0,3000.0
5,3000.0,-0.0


In [27]:
nu = pypsa.Network(snapshots=range(12))

nu.add("Bus", "bus")

# Can get bad interactions if SU > RU and p_min_pu; similarly if SD > RD
nu.add(
    "Generator",
    "coal",
    bus="bus",
    marginal_cost=20,
    committable=True,
    p_min_pu=0.05,
    initial_status=0,
    ramp_limit_start_up=0.1,
    ramp_limit_up=0.2,
    ramp_limit_down=[0.25, 0.2, 0.15, 0.4, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
    ramp_limit_shut_down=[0.2, 0.15, 0.1, 0.3, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
    p_nom=10000.0,
)

nu.add("Generator", "gas", bus="bus", marginal_cost=70, p_nom=10000)

nu.add("Load", "load", bus="bus", p_set=[0.0, 200.0, 7000, 7000, 7000, 2000, 500, 500, 500, 300, 300, 1000])

In [28]:
nu.optimize(linearized_unit_commitment=True)

Index(['bus'], dtype='object', name='Bus')
Index(['bus'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.03s
INFO:linopy.solvers:Log file at /tmp/highs.log.
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 60 primals, 139 duals
Objective: 1.04e+06
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-com-p-lower, Generator-com-p-upper, Generator-com-transition-start-up, Generator-com-transition-shut-down, Generator-com-status-min_up_time_must_stay_up, Generator-com-p-before, Generator-com-p-current, Generator-com-partly-start-up, Generator-com-partly-shut-down, Generator-com-p-ramp_limit_up, Generator-com-p-ramp_limit_down were not assigned to the network.


Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d-dirty]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
102 rows, 35 cols, 335 nonzeros
83 rows, 32 cols, 286 nonzeros
83 rows, 32 cols, 286 nonzeros
Presolve : Reductions: rows 83(-56); columns 32(-28); elements 286(-111)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
         15     1.0360000000e+06 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 15
Objective value     :  1.0360000000e+06
HiGHS run time      :          0.00


('ok', 'optimal')

In [43]:
nu.generators_t.ramp_limit_down - nu.generators_t.ramp_limit_shut_down

Generator,coal
snapshot,
0,0.05
1,0.05
2,0.05
3,0.10
4,0.30
5,0.30
6,0.30
7,0.30
8,0.30


In [39]:
nu.model.constraints["Generator-com-p-ramp_limit_down"]

Constraint `Generator-com-p-ramp_limit_down` (snapshot: 11, Generator-com: 1) - 11 masked entries:
--------------------------------------------------------------------------------------------------
[1, coal]: None
[2, coal]: None
[3, coal]: None
[4, coal]: None
[5, coal]: None
[6, coal]: None
[7, coal]: None
[8, coal]: None
[9, coal]: None
[10, coal]: None
[11, coal]: None

In [35]:
2400*nu.generators_t.status+100*nu.generators_t.status.shift(1)

Generator-com,coal
snapshot,
0,NaN
1,960.000000
2,2440.000000
3,2500.000000
4,2500.000000
5,1380.000000
6,400.000000
7,134.444444
8,125.000000


In [36]:
nu.generators_t.status

Generator-com,coal
snapshot,
0,0.000000
1,0.400000
2,1.000000
3,1.000000
4,1.000000
5,0.533333
6,0.144444
7,0.050000
8,0.050000


In [32]:
nu.generators_t.p

Generator,coal,gas
snapshot,,
0,-0.0,-0.0
1,200.0,-0.0
2,1600.0,5400.0
3,3600.0,3400.0
4,5600.0,1400.0
5,2000.0,-0.0
6,500.0,-0.0
7,500.0,-0.0
8,500.0,-0.0


In [33]:
stop

NameError: name 'stop' is not defined

In [ ]:
nu.generators.ramp_limit_shut_down

Generator
coal    0.015
gas     1.000
Name: ramp_limit_shut_down, dtype: float64

In [ ]:
nu.generators_t.p

Generator,coal,gas
snapshot,,
0,0.0,0.0
1,200.0,0.0
2,1600.0,5400.0
3,3600.0,3400.0
4,4000.0,3000.0
5,1500.0,500.0
6,0.0,0.0


In [ ]:
nu.generators_t.status

Generator-com,coal
snapshot,
0,0.0
1,0.4
2,1.0
3,1.0
4,1.0
5,1.0
6,0.0


In [ ]:
nu.generators.loc["coal"]

attribute
bus                            bus
control                         PQ
type                              
p_nom                      10000.0
p_nom_mod                      0.0
p_nom_extendable             False
p_nom_min                      0.0
p_nom_max                      inf
p_min_pu                       0.3
p_max_pu                       1.0
p_set                          0.0
q_set                          0.0
sign                           1.0
carrier                           
marginal_cost                 20.0
marginal_cost_quadratic        0.0
build_year                       0
lifetime                       inf
capital_cost                   0.0
efficiency                     1.0
committable                   True
start_up_cost                200.0
shut_down_cost               150.0
stand_by_cost                  0.0
min_up_time                      3
min_down_time                    2
up_time_before                   1
down_time_before                 0
ramp_limit

## Rolling horizon example

This example solves sequentially in batches

In [ ]:
sets_of_snapshots = 6
p_set = [4000, 5000, 700, 800, 4000]

nu = pypsa.Network(snapshots=range(len(p_set) * sets_of_snapshots))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    committable=True,
    p_min_pu=0.3,
    marginal_cost=20,
    min_down_time=2,
    min_up_time=3,
    up_time_before=1,
    ramp_limit_up=1,
    ramp_limit_down=1,
    ramp_limit_start_up=1,
    ramp_limit_shut_down=1,
    shut_down_cost=150,
    start_up_cost=200,
    p_nom=10000,
)

nu.add(
    "Generator",
    "gas",
    bus="bus",
    committable=True,
    marginal_cost=70,
    p_min_pu=0.1,
    up_time_before=2,
    min_up_time=3,
    shut_down_cost=20,
    start_up_cost=50,
    p_nom=1000,
)

nu.add("Load", "load", bus="bus", p_set=p_set * sets_of_snapshots)

In [ ]:
overlap = 2
for i in range(sets_of_snapshots):
    nu.optimize(nu.snapshots[i * len(p_set) : (i + 1) * len(p_set) + overlap])

INFO:linopy.model: Solve problem using Glpk solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.solvers:GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --lp /tmp/linopy-problem-i08de5gv.lp --output /tmp/linopy-solve-v7qf865e.sol
Reading problem data from '/tmp/linopy-problem-i08de5gv.lp'...
96 rows, 56 columns, 253 non-zeros
42 integer variables, all of which are binary
646 lines were read
GLPK Integer Optimizer 5.0
96 rows, 56 columns, 253 non-zeros
42 integer variables, all of which are binary
Preprocessing...
3 hidden packing inequaliti(es) were detected
23 rows, 20 columns, 58 non-zeros
15 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+03  ratio =  1.000e+03
GM: min|aij| =  5.623e-01  max|aij| =  1.778e+00  ratio =  3.162e+00
EQ: min|aij| =  3.162e-01  max|aij| =  1.000e+00  ratio =  3.162e+00
2N: min|aij| =  1.953e-01  max|aij| =  1.000e+00  ratio =  5.120e+00
Constructing initial basis...
Size of 

In [ ]:
pd.concat(
    {"Active": nu.generators_t.status.astype(bool), "Output": nu.generators_t.p}, axis=1
)

Active         Output       
Generator-com   coal    gas    coal    gas
snapshot                                  
0               True   True  3900.0  100.0
1               True   True  4900.0  100.0
2              False   True     0.0  700.0
3              False   True     0.0  800.0
4               True  False  4000.0    0.0
5               True  False  4000.0    0.0
6               True  False  5000.0    0.0
7              False   True     0.0  700.0
8              False   True     0.0  800.0
9               True   True  3900.0  100.0
10              True  False  4000.0    0.0
11              True  False  5000.0    0.0
12             False   True     0.0  700.0
13             False   True     0.0  800.0
14              True   True  3900.0  100.0
15              True  False  4000.0    0.0
16              True  False  5000.0    0.0
17             False   True     0.0  700.0
18             False   True     0.0  800.0
19              True   True  3900.0  100.0
20              True  False  4000.0    0.0
21              True  False  5000.0    0.0
22             False   True     0.0  700.0
23             False   True     0.0  800.0
24              True   True  3900.0  100.0
25              True  False  4000.0    0.0
26              True  False  5000.0    0.0
27             False   True     0.0  700.0
28             False   True     0.0  800.0
29              True   True  3900.0  100.0